Importing the libraries

In [ ]:
import pandas as pd
import numpy as np

Logic for function to check if dataframe coloumn contains date or not


In [ ]:
 dates = pd.date_range('20130101', periods=6)
 df = pd.DataFrame(np.random.randn(6, 4), columns=list('ABCD'))
rowIndex = df.index[3]
rowIndex

3

In [ ]:
col_one_list = df.iloc[:,1].values.tolist() #coverting to list just checking something
col_one_list

[-0.6213726566223546,
 -1.0466555614819182,
 0.7834799521768309,
 -0.23579212861147522,
 -0.26090125703574396,
 0.1755380771515636]

In [ ]:
import random

The logic I'm trying to implement here is that we take 3 random values with the col of the dataframe and check for each of them if the date is in one of the given format that I hypothesised, now the reasoning behind taking 3 cols was that it would save time, realistically we could do a lot more but if 3 random values are not the date value, Statistically and the probablity is that a majority of the values are not the date value hence we would probably not use that feature and drop it anyways

#**TASK** **1**

In [ ]:
def IsDateInCol(df):
  number_of_cols = df.shape[1]
  number_of_rows = df.shape[0]
  cols_with_date=[]
  position_of_day ={}  
  ''' initializing a dictionary to know at what index 
    does the day start because that can be different according to different formats of date'''
  for i in range(number_of_cols):
    flag = 0   # Gets raised if we gate a positive result for date
    col_list=df.iloc[:,i].values.tolist()
    random_number1 = random.randint(0,len(col_list)-1)
    random_number2 = random.randint(0,len(col_list)-1)
    random_number3 = random.randint(0,len(col_list)-1)
    list_of_randoms = [random_number1, random_number2, random_number3]
    if df.iloc[:,i].dtype == "datetime64[ns]":
      flag=1
      position_of_day[i] = 10 #random value which is gonna indicate that I have to convert to str first
    for j in list_of_randoms:
      if isinstance(col_list[j], str):
        #2013-01-01 few sample date formats we should check for
        #01-01-2013
        #2013/01/01
        #01/01/2013
        #01/01/13
        #01-01-13
        #1st January 2013??????
        # check for case 1
        current_element =col_list[j]
        if len(col_list[j])==10:
          if (current_element[4]=="-" or current_element[4]=="/") and (current_element[7]=="-" or current_element[7]=="/"):
            try:
              year = int(current_element[0:4])
              if isinstance(year,int):
                flag=1
                position_of_day[i] = 5
            except:
              pass
        if len(col_list[j])==10:
          if (current_element[2]=="-" or current_element[2]=="/") and (current_element[7]=="-" or current_element[5] == "/"):
            try:
              year = int(current_element[6:10])
              if isinstance(year,int):
                flag=1
                position_of_day[i] = 0 
            except:
              pass
        if len(current_element)==8:
          if (current_element[2]=="-" or current_element[2]=="/") and (current_element[5]=="-" or current_element[5] == "/"):
            try:
              year = int(current_element[0:2])
              month = int(current_element[3:5])
              if isinstance(year,int) and isinstance(month, int):
                flag=1
                position_of_day[i] = 0
            except:
              pass
    #if flag== 1:
      #cols_with_date.append(i)
  return position_of_day

        



This function should be robust enough to go through most dataframes unless the number of dates in that dataframe are in minority, which like I said before would more than likely cause us to drop the coloumn during the EDA process itself

#Testing

An AI approach could also be applied to this where we can do text analysis to check for months like January 24th and even check for those values. Although I'm not sure if that would be better as that would be computationally very expensive

In [ ]:
df2 = pd.DataFrame({'A': 1.,
                       'B': pd.Timestamp('20130102'),
                       'C': pd.Series(1, index=list(range(4)), dtype='float32'),
                        'D': np.array([3] * 4, dtype='int32'),
                        'E': pd.Categorical(["sw17-04-03", "as17-23-12", "2017-09-01", "2017-09-12"]),
                     'F': 'foo'})
df2

,A,B,C,D,E,F
0,1.0,2013-01-02,1.0,3,sw17-04-03,foo
1,1.0,2013-01-02,1.0,3,as17-23-12,foo
2,1.0,2013-01-02,1.0,3,2017-09-01,foo
3,1.0,2013-01-02,1.0,3,2017-09-12,foo


In [ ]:
IsDateInCol(df2)

{1: 10, 4: 5}

In [ ]:
list(IsDateInCol(df2).keys())

[1, 4]

#**TASK** **1.2**

In [ ]:
import itertools



2

In [ ]:
def SubOfDateCols(df):
  date_cols_dictionary = IsDateInCol(df)
  list_of_cols_withDate = list(date_cols_dictionary.keys())
  number_of_rows = df.shape[0]
  # we need to make coloumns which will be equal number_of_cols combinations of 2
  #nC2 combinations
  
  for i in range(number_of_rows):
    list_of_days=[]
    for j in list_of_cols_withDate:
    #Lets get all the values of days in list format
      index = date_cols_dictionary[j]
      if index==10:
        date_value = df.iloc[:,j].values[i]
        date_str = str(date_value)
        day = date_str[5:7]
        try:
          day = int(day)
          if isinstance(day,int):
            list_of_days.append(day)
        except:
          pass
      if index == 5:
        date_value = df.iloc[:,j].values[i]
        date_str = str(date_value)
        day = date_str[5:7]
        try:
          day = int(day)
          if isinstance(day,int):
            list_of_days.append(day)
        except: 
          pass
  

      if index == 0:
        date_value = df.iloc[:,j].values[i]
        date_str = str(date_value)
        day = date_str[0:2]
        try:
          day = int(day)
          if isinstance(day,int):
            list_of_days.append(day)
        except:
          pass

    # Now we have a list of all days we need to make combinations and add them
    # as new coloumns in the dataframe
    combination =list(itertools.combinations(list_of_days,2))
    for k in range(len(combination)):
      new_col_title = "a" + str(k)
      df.loc[i, new_col_title] = combination[k][0]-combination[k][1]
  return df


    

#TESTING

In [ ]:
df = pd.DataFrame({'A': 1.,
                       'B': pd.Timestamp('20130102'),
                       'C': pd.Series(1, index=list(range(4)), dtype='float32'),
                        'D': np.array([3] * 4, dtype='int32'),
                        'E': pd.Categorical(["sw17-04-03", "as17-23-12", "2017-09-01", "2017-09-12"]),
                     'F': 'foo'})
df

,A,B,C,D,E,F
0,1.0,2013-01-02,1.0,3,sw17-04-03,foo
1,1.0,2013-01-02,1.0,3,as17-23-12,foo
2,1.0,2013-01-02,1.0,3,2017-09-01,foo
3,1.0,2013-01-02,1.0,3,2017-09-12,foo


In [ ]:
SubOfDateCols(df)


,A,B,C,D,E,F,a0
0,1.0,2013-01-02,1.0,3,sw17-04-03,foo,-3.0
1,1.0,2013-01-02,1.0,3,as17-23-12,foo,-22.0
2,1.0,2013-01-02,1.0,3,2017-09-01,foo,-8.0
3,1.0,2013-01-02,1.0,3,2017-09-12,foo,-8.0


In [ ]:
date_cols_dictionary = IsDateInCol(df)
list_of_cols_withDate = list(date_cols_dictionary.keys())
number_of_rows = df.shape[0]
# we need to make coloumns which will be equal number_of_cols combinations of 2
#nC2 combinations
list_of_days=[]
for i in range(number_of_rows):
  
  for j in list_of_cols_withDate:
  #Lets get all the values of days in list format
    index = date_cols_dictionary[j]
    if index==10:
      date_value = df.iloc[:,j].values[i]
      date_str = str(date_value)
      day = date_str[5:7]
      try:
        day = int(day)
        if isinstance(day,int):
          list_of_days.append(day)
      except:
        pass
    if index == 5:
      date_value = df.iloc[:,j].values[i]
      date_str = str(date_value)
      day = date_str[5:7]
      try:
        day = int(day)
        if isinstance(day,int):
          list_of_days.append(day)
      except: 
        pass

    if index == 0:
      date_value = df.iloc[:,j].values[i]
      date_str = str(date_value)
      day = date_str[0:2]
      try:
        day = int(day)
        if isinstance(day,int):
          list_of_days.append(day)
      except:
        pass
print(list_of_days)


[1, 4, 1, 23, 1, 9, 1, 9]


#TASK 1.3

In [ ]:
def drop_except_new(df):
  number_of_cols_orignally = df.shape[1]
  df = SubOfDateCols(df) #we get the new dataframe
  for i in range(number_of_cols_orignally-1):
    df = df.drop(df.columns[0], axis=1)
  return df

#Testing

In [ ]:
df = pd.DataFrame({'A': 1.,
                       'B': pd.Timestamp('20130102'),
                       'C': pd.Series(1, index=list(range(4)), dtype='float32'),
                        'D': np.array([3] * 4, dtype='int32'),
                        'E': pd.Categorical(["sw17-04-03", "as17-23-12", "2017-09-01", "2017-09-12"]),
                     'F': 'foo'})
df

,A,B,C,D,E,F
0,1.0,2013-01-02,1.0,3,sw17-04-03,foo
1,1.0,2013-01-02,1.0,3,as17-23-12,foo
2,1.0,2013-01-02,1.0,3,2017-09-01,foo
3,1.0,2013-01-02,1.0,3,2017-09-12,foo


In [ ]:
drop_except_new(df)

,a0
0,-3.0
1,-22.0
2,-8.0
3,-8.0


#Pearson Correlation

In [ ]:
import scipy
from scipy.stats.stats import pearsonr

In [ ]:
pearson_drop(df):
  columns = df.shape[1]
  list_of_cols_to_drop = []
  for i in range(int(columns)-1):
    for j in range(i, int(columns)-1) 
      if pearsonr(df.columns[j],df.columns[j+1])>0.85:
        list_of_cols_to_drop.append(j+1) # get all combinations of cols
  mylist = list(dict.fromkeys(list_of_cols_to_drop)) # remove duplicates
  df=df.drop(df.columns[mylist], axis = 1, inplace = True) 
  return df

      

#TASK 3

Any kind of preprocessing or as stated in the instance should be done on the training data only because we are using the test data to build the model for the future, so we should not impute the test data as if we rish overfitting the model and it is not really test data if we let our training data affect it by changing missing values etc etc.

#TASK 4

How do we speed up python code?


There are quite a few ways of doing this. First and most obvious one would be-

1.   Use a more efficient algorithm
2.   Use a data structure

for instance a lot of stuff can be done by brute force method going through every possible combination but can often cause time complexities of O(N^3) or more. We might want to solve the problem in linear time or nLog(n) time. For that a good understanding of algorithms and creative thinking is important.

Other and more easier way to do it is by using a data structure for instance using an unordered hash set or map(dict or set in python) which have more efficient look ups than standard arrays. Although I have a bad habit of using lists till I come across a problem which has to solved in a specific time limit..... 

